# Анализ результатов моделей

In [1]:
import pandas as pd
import sys
import os

# Добавляем путь к проекту, чтобы можно было импортировать из utils
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()
    
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, "..", ".."))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)
from utils.config import AGENT_RESULTS_DIR, EXPERIMENTS_DIR, RELEVANCE_COL  
from utils.inspector import inspect_row
from utils.unify_columns import unify_df

In [2]:
df_baseline_val = pd.read_csv(os.path.join(EXPERIMENTS_DIR, "baseline_val_predictions.csv"))
df_val1 = pd.read_csv(os.path.join(AGENT_RESULTS_DIR, "agent_val_predictions_v1.csv"))
df_val2 = pd.read_csv(os.path.join(AGENT_RESULTS_DIR, "agent_val_predictions_v2.csv"))
df_val3 = pd.read_csv(os.path.join(AGENT_RESULTS_DIR, "agent_val_predictions_v3.csv"))
df_baseline_test = pd.read_csv(os.path.join(EXPERIMENTS_DIR, "baseline_test_predictions.csv"))
df_test1 = pd.read_csv(os.path.join(AGENT_RESULTS_DIR, "agent_test_predictions_v1.csv"))
df_test2 = pd.read_csv(os.path.join(AGENT_RESULTS_DIR, "agent_test_predictions_v2.csv"))
df_test3 = pd.read_csv(os.path.join(AGENT_RESULTS_DIR, "agent_test_predictions_v3.csv"))

In [3]:
# Применяем ко всем датасетам
for df_name in ['df_val1', 'df_val2', 'df_val3', 'df_baseline_val', 'df_test1', 'df_test2', 'df_test3', 'df_baseline_test']:
    globals()[df_name] = unify_df(globals()[df_name])

for df_name in ['df_val1', 'df_val2', 'df_val3', 'df_baseline_val', 'df_test1', 'df_test2', 'df_test3', 'df_baseline_test']:
    df = globals()[df_name]
    if 'model_response' in df.columns:
        globals()[df_name] = df.drop(columns=['model_response'])

In [4]:
exclude_cols = ['agent_log', 'pred_relevance']

def clean_df(df):
    return df.drop(columns=exclude_cols, errors='ignore').fillna('nan')

assert all(
    clean_df(df_val1).equals(clean_df(df))
    for df in [df_val2, df_val3, df_baseline_val]
), "Валидационные датасеты не идентичны"

assert all(
    clean_df(df_test1).equals(clean_df(df))
    for df in [df_test2, df_test3, df_baseline_test]
), "Тестовые датасеты не идентичны"

In [5]:
shapes = {
    "размер валидации": df_baseline_val.shape,
    "размер теста": df_baseline_test.shape,
}

shape_df = pd.DataFrame(shapes, index=["rows", "columns"]).T
shape_df

,rows,columns
размер валидации,299,10
размер теста,500,10


In [6]:
dfs = {
    "распределение таргета на валидации": df_baseline_val,
    "распределение таргета на тесте": df_test1,
}

balance_table = pd.DataFrame({
    name: df['relevance_new'].value_counts(normalize=True)
    for name, df in dfs.items()
}).fillna(0).T

balance_table = (balance_table * 100).round(2)
balance_table

relevance_new,1.0,0.0
распределение таргета на валидации,57.53,42.47
распределение таргета на тесте,63.40,36.60


In [7]:
pred_balance_table = pd.DataFrame({
    name: df['pred_relevance'].value_counts(normalize=True)
    for name, df in dfs.items()
}).fillna(0).T

pred_balance_table = (pred_balance_table * 100).round(2)
pred_balance_table

pred_relevance,1.0,0.0
распределение таргета на валидации,66.22,33.78
распределение таргета на тесте,70.20,29.80


In [8]:
metrics = [
    ("val", "baseline", df_baseline_val, "pred_relevance", "gpt + few-shot in-context learning prompt"),
    ("val", "agent1", df_val1, "pred_relevance", "агент 1 версия"),
    ("val", "agent2", df_val2, "pred_relevance", "агент 2 версия"),
    ("val", "agent3", df_val3, "pred_relevance", "агент 3 версия"),
    ("test", "baseline", df_baseline_test, "pred_relevance", "gpt + few-shot in-context learning prompt"),
    ("test", "agent1", df_test1, "pred_relevance", "агент 1 версия"),
    ("test", "agent2", df_test2, "pred_relevance", "агент 2 версия"),
    ("test", "agent3", df_test3, "pred_relevance", "агент 3 версия"),
]

rows = []

for split, model, df, pred_col, desc in metrics:
    acc = (df[RELEVANCE_COL] == df[pred_col]).mean()
    rows.append({"split": split, "model": model, "description": desc, "accuracy": round(acc, 4)})

acc_df = pd.DataFrame(rows)
print('=== Сравнение бейзлайна и версий агента ===\n')
acc_df

=== Сравнение бейзлайна и версий агента ===



,split,model,description,accuracy
0,val,baseline,gpt + few-shot in-context learning prompt,0.6856
1,val,agent1,агент 1 версия,0.6957
2,val,agent2,агент 2 версия,0.6221
3,val,agent3,агент 3 версия,0.6388
4,test,baseline,gpt + few-shot in-context learning prompt,0.7840
5,test,agent1,агент 1 версия,0.7640
6,test,agent2,агент 2 версия,0.7360
7,test,agent3,агент 3 версия,0.7440


In [9]:
def evaluate_models(df_baseline, df_agent1, df_agent2, df_agent3=None, name='val'):
    id_col = 'permalink'
    label_col = RELEVANCE_COL
    pred_col = 'pred_relevance'

    # Убедимся, что индекс уникальный
    for df in (df_baseline, df_agent1, df_agent2) + ((df_agent3,) if df_agent3 is not None else ()):
        if not df.index.is_unique:
            df.reset_index(drop=True, inplace=True)

    # Проверка совпадения id и меток
    for df in (df_agent1, df_agent2) + ((df_agent3,) if df_agent3 is not None else ()):
        assert (df[id_col].reset_index(drop=True) == df_baseline[id_col].reset_index(drop=True)).all(), f'{name}: ID не совпадают'
        assert (df[label_col].reset_index(drop=True) == df_baseline[label_col].reset_index(drop=True)).all(), f'{name}: метки не совпадают'

    df = df_baseline[[id_col, label_col]].copy()
    df['baseline_pred'] = df_baseline[pred_col].values
    df['agent1_pred'] = df_agent1[pred_col].values
    df['agent2_pred'] = df_agent2[pred_col].values
    if df_agent3 is not None:
        df['agent3_pred'] = df_agent3[pred_col].values

    # Ошибки предсказания
    df['baseline_error'] = df['baseline_pred'] != df[label_col]
    df['agent1_error'] = df['agent1_pred'] != df[label_col]
    df['agent2_error'] = df['agent2_pred'] != df[label_col]
    if df_agent3 is not None:
        df['agent3_error'] = df['agent3_pred'] != df[label_col]

    # Консенсус ошибок (все ошибаются и предсказания совпадают)
    if df_agent3 is not None:
        df['consensus'] = (
            df['baseline_error'] &
            df['agent1_error'] &
            df['agent2_error'] &
            df['agent3_error'] &
            (df['baseline_pred'] == df['agent1_pred']) &
            (df['baseline_pred'] == df['agent2_pred']) &
            (df['baseline_pred'] == df['agent3_pred'])
        )
    else:
        df['consensus'] = (
            df['baseline_error'] &
            df['agent1_error'] &
            df['agent2_error'] &
            (df['baseline_pred'] == df['agent1_pred']) &
            (df['baseline_pred'] == df['agent2_pred'])
        )

    # Ошибки только одного агента
    if df_agent3 is not None:
        df['only_baseline_wrong'] = df['baseline_error'] & ~df['agent1_error'] & ~df['agent2_error'] & ~df['agent3_error']
        df['only_agent1_wrong'] = ~df['baseline_error'] & df['agent1_error'] & ~df['agent2_error'] & ~df['agent3_error']
        df['only_agent2_wrong'] = ~df['baseline_error'] & ~df['agent1_error'] & df['agent2_error'] & ~df['agent3_error']
        df['only_agent3_wrong'] = ~df['baseline_error'] & ~df['agent1_error'] & ~df['agent2_error'] & df['agent3_error']
    else:
        df['only_baseline_wrong'] = df['baseline_error'] & ~df['agent1_error'] & ~df['agent2_error']
        df['only_agent1_wrong'] = ~df['baseline_error'] & df['agent1_error'] & ~df['agent2_error']
        df['only_agent2_wrong'] = ~df['baseline_error'] & ~df['agent1_error'] & df['agent2_error']

    total = len(df)
    summary = {
        'baseline_errors': df['baseline_error'].sum(),
        'agent1_errors': df['agent1_error'].sum(),
        'agent2_errors': df['agent2_error'].sum(),
        'consensus_errors': df['consensus'].sum(),
        'consensus_fraction': df['consensus'].mean(),
        'only_baseline_wrong': df['only_baseline_wrong'].sum(),
        'only_agent1_wrong': df['only_agent1_wrong'].sum(),
        'only_agent2_wrong': df['only_agent2_wrong'].sum(),
    }
    if df_agent3 is not None:
        summary['agent3_errors'] = df['agent3_error'].sum()
        summary['only_agent3_wrong'] = df['only_agent3_wrong'].sum()

    print(f'\n{name.upper()} SET')
    print(f"Total samples: {total}")
    print(f"Baseline errors: {summary['baseline_errors']}")
    print(f"Agent1 errors: {summary['agent1_errors']}")
    print(f"Agent2 errors: {summary['agent2_errors']}")
    if df_agent3 is not None:
        print(f"Agent3 errors: {summary['agent3_errors']}")
    print(f"Consensus errors: {summary['consensus_errors']} ({summary['consensus_fraction']:.2%})")
    print(f"Only baseline wrong: {summary['only_baseline_wrong']}")
    print(f"Only agent1 wrong: {summary['only_agent1_wrong']}")
    print(f"Only agent2 wrong: {summary['only_agent2_wrong']}")
    if df_agent3 is not None:
        print(f"Only agent3 wrong: {summary['only_agent3_wrong']}")

    save_dir = os.path.join(CURRENT_DIR, 'analysis_errors', name)
    os.makedirs(save_dir, exist_ok=True)

    def save_rows(mask, source_df, suffix):
        idxs = df[mask].index
        full = source_df.loc[idxs].copy()
        full.to_csv(os.path.join(save_dir, f'{suffix}.csv'), index=False)

    consensus_idxs = df[df['consensus']].index
    consensus_rows = df_baseline.loc[consensus_idxs].copy()
    consensus_rows.to_csv(os.path.join(save_dir, 'consensus_errors.csv'), index=False)

    save_rows(df['only_baseline_wrong'], df_baseline, 'only_baseline_wrong')
    save_rows(df['only_agent1_wrong'], df_agent1, 'only_agent1_wrong')
    save_rows(df['only_agent2_wrong'], df_agent2, 'only_agent2_wrong')
    if df_agent3 is not None:
        save_rows(df['only_agent3_wrong'], df_agent3, 'only_agent3_wrong')

    for model_name, source_df in zip(
        ['baseline', 'agent1', 'agent2'] + (['agent3'] if df_agent3 is not None else []),
        [df_baseline, df_agent1, df_agent2] + ([df_agent3] if df_agent3 is not None else [])
    ):
        save_rows(df['consensus'], source_df, f'{model_name}_consensus_error')

    ret = {
        'name': name,
        'summary': summary,
        'df': df,
        'consensus_errors': df[df['consensus']].copy(),
        'only_baseline_wrong': df[df['only_baseline_wrong']].copy(),
        'only_agent1_wrong': df[df['only_agent1_wrong']].copy(),
        'only_agent2_wrong': df[df['only_agent2_wrong']].copy(),
    }
    if df_agent3 is not None:
        ret['only_agent3_wrong'] = df[df['only_agent3_wrong']].copy()

    return ret


def evaluate_models_both(
    df_baseline_val, df_val1, df_val2, df_val3,
    df_baseline_test, df_test1, df_test2, df_test3
):
    res_val = evaluate_models(df_baseline_val, df_val1, df_val2, df_val3, name='val')
    res_test = evaluate_models(df_baseline_test, df_test1, df_test2, df_test3, name='test')
    return res_val, res_test

In [10]:
val_results, test_results = evaluate_models_both(
    df_baseline_val, df_val1, df_val2, df_val3,
    df_baseline_test, df_test1, df_test2, df_test3
)


VAL SET
Total samples: 299
Baseline errors: 94
Agent1 errors: 91
Agent2 errors: 113
Agent3 errors: 108
Consensus errors: 68 (22.74%)
Only baseline wrong: 1
Only agent1 wrong: 3
Only agent2 wrong: 8
Only agent3 wrong: 9

TEST SET
Total samples: 500
Baseline errors: 108
Agent1 errors: 118
Agent2 errors: 132
Agent3 errors: 128
Consensus errors: 66 (13.20%)
Only baseline wrong: 11
Only agent1 wrong: 17
Only agent2 wrong: 9
Only agent3 wrong: 13


In [11]:
val_baseline = val_results["consensus_errors"]["baseline_pred"].value_counts()
test_baseline = test_results["consensus_errors"]["baseline_pred"].value_counts()
summary = pd.DataFrame({
    "Val - Baseline": val_baseline,
    "Test - Baseline": test_baseline
}).fillna(0).astype(int).sort_index()
summary = summary.reset_index().rename(columns={"baseline_pred": "predictions"})
print("===Распределение предсказаний по консенсусным ошибкам (где все модели одинаково ошибаются)===\n")
summary

===Распределение предсказаний по консенсусным ошибкам (где все модели одинаково ошибаются)===



,predictions,Val - Baseline,Test - Baseline
0,0.0,21,25
1,1.0,47,41


### Анализ ошибок агента 3

In [12]:
file_path = os.path.join(CURRENT_DIR, "analysis_errors", "val", "only_agent3_wrong.csv")
unique_error_agent3 = pd.read_csv(file_path)
unique_error_agent3.shape

(9, 11)

In [13]:
assessors_errors_3 = [] 
model_errors_3 = []

In [14]:
model_errors_3.append(int(unique_error_agent3.loc[0]['permalink']))

In [15]:
inspect_row(unique_error_agent3, idx=0)

### Модель упустила важный модификатор в запросе

не учтен формат ресторана, упоминание стейка с палтусом в отзыве становится релевантным фактором для модели

In [16]:
model_errors_3.append(int(unique_error_agent3.loc[1]['permalink']))

In [17]:
inspect_row(unique_error_agent3, idx=1)

In [18]:
model_errors_3.append(int(unique_error_agent3.loc[2]['permalink']))

In [19]:
inspect_row(unique_error_agent3, idx=2)

### Латентная релевантность раскрыта агентом

Сработал внешний поиск

In [20]:
assessors_errors_3.append(int(unique_error_agent3.loc[3]['permalink']))

In [21]:
inspect_row(unique_error_agent3, idx=3)

In [22]:
model_errors_3.append(int(unique_error_agent3.loc[4]['permalink']))

In [23]:
inspect_row(unique_error_agent3, idx=4)

### Латентная релевантность раскрыта агентом

In [24]:
assessors_errors_3.append(int(unique_error_agent3.loc[5]['permalink']))

In [25]:
inspect_row(unique_error_agent3, idx=5)

Всё-таки какая-то релевантность есть, кофе на вынос они предлагают...

In [26]:
assessors_errors_3.append(int(unique_error_agent3.loc[6]['permalink']))

In [27]:
inspect_row(unique_error_agent3, idx=6)

In [28]:
model_errors_3.append(int(unique_error_agent3.loc[7]['permalink']))

In [29]:
inspect_row(unique_error_agent3, idx=7)

Не учтен формат заведения "поликлиника - клиника"

In [30]:
model_errors_3.append(int(unique_error_agent3.loc[8]['permalink']))

In [31]:
inspect_row(unique_error_agent3, idx=8)

In [32]:
total_unique = len(unique_error_agent3)
num_model_errors = len(model_errors_3)
num_assessor_errors = len(assessors_errors_3)

print(f"Всего уникальных ошибок агента 3: {total_unique}")
print(f"Из них модельных ошибок: {num_model_errors} ({num_model_errors / total_unique:.1%})")
print(f"Ошибок аннотаций: {num_assessor_errors} ({num_assessor_errors / total_unique:.1%})")

# Проверка: пересекаются ли model_errors и assessors_errors (не должно быть)
intersection = set(model_errors_3) & set(assessors_errors_3)
if intersection:
    print(f"⚠️ Пересечения в списках model_errors и assessors_errors: {len(intersection)}")

Всего уникальных ошибок агента 3: 9
Из них модельных ошибок: 6 (66.7%)
Ошибок аннотаций: 3 (33.3%)


# Анализ консенсусных ошибок на валидации

In [33]:
model_consensus_errors = []
assessors_errors_consensus = []

In [34]:
file_path = os.path.join(CURRENT_DIR, "analysis_errors", "val", "agent3_consensus_error.csv")
agent3_consensus_error = pd.read_csv(file_path)
f'Количество примеров на валидации, в которых ошибаются и агенты и бейзлайн: {agent3_consensus_error.shape[0]}'

'Количество примеров на валидации, в которых ошибаются и агенты и бейзлайн: 68'

Ошибка асессора. Elasti_city. Студия растяжки и фитнеса. ул Кастанаевская, д 39. Модель распознает по названию.

In [35]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[0]['permalink']))

In [36]:
inspect_row(agent3_consensus_error, idx=0)

### Модель упустила важный модификатор в запросе

Похоже, модель посчитала совпадение по адресу и тематике достаточно весомым, не выявила, что организация не удовлетворяет требование дежурности.

Возможно, стоит добавить патч в промт и явно учитывать ключевые уточнения в запросе — например, "дежурная", "круглосуточная", "24 часа" — и требовать подтверждения этих условий в описании/отзывах.

In [37]:
model_consensus_errors.append(int(agent3_consensus_error.loc[1]['permalink']))

In [38]:
inspect_row(agent3_consensus_error, idx=1)

### Малоинформативный/общий запрос пользователя

В запросе «центр рязань» слово «центр» очень общее, а рубрика организации — «Медцентр, клиника». +  
В запросе есть «рязань», адрес организации тоже в Рязани — это усилило уверенность модели, что организация релевантна.

In [39]:
model_consensus_errors.append(int(agent3_consensus_error.loc[2]['permalink']))

In [40]:
inspect_row(agent3_consensus_error, idx=2)

### Справочный/агрегирующий запрос

В целом указан зоомагазин. Скорее ошибка асессора. Точно какая-то релевантность есть!

In [41]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[3]['permalink']))

In [42]:
inspect_row(agent3_consensus_error, idx=3)

### Модель упустила важный модификатор в запросе

Может, необходимо уточнить промт и в явном виде подсветить модели правило о формате услуге.


In [43]:
model_consensus_errors.append(int(agent3_consensus_error.loc[4]['permalink']))

In [44]:
inspect_row(agent3_consensus_error, idx=4)

### асессор не учитывает локацию в запросе. 
Ошибка асессора. 

In [45]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[5]['permalink']))

In [46]:
inspect_row(agent3_consensus_error, idx=5)

In [47]:
model_consensus_errors.append(int(agent3_consensus_error.loc[6]['permalink']))

In [48]:
inspect_row(agent3_consensus_error, idx=6)

In [49]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[7]['permalink']))

In [50]:
inspect_row(agent3_consensus_error, idx=7)

### Малоинформативный/общий запрос пользователя
Тут совпали и географический признак и в целом в рубрику попадает. Скорее ошибка асессора, на мой взгляд, какой-то процент релевантности точно есть

In [51]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[8]['permalink']))

In [52]:
inspect_row(agent3_consensus_error, idx=8)

Высокие цены точно отмечены в отзыве - модель не игнорирует

In [53]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[9]['permalink']))

In [54]:
inspect_row(agent3_consensus_error, idx=9)

### Модель упустила важный модификатор в запросе

Как в случае с "обучением женских стрижек" тут модель проигнорировала модификатор "выкуп"

In [55]:
model_consensus_errors.append(int(agent3_consensus_error.loc[10]['permalink']))

In [56]:
inspect_row(agent3_consensus_error, idx=10)

В поисковике есть указание на страны, а значит релевантность точно не нулевая... Запрос удовлетворяет организации при перепроверки

In [57]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[11]['permalink']))

In [58]:
inspect_row(agent3_consensus_error, idx=11)

### Модель упустила важный модификатор 
возможно, тут нужно сначала парсить запрос, искать в нем ключевые слова (круглосуточный, 24 часа...) и менять запрос в поисковик, чтобы точно искать часы работы

In [59]:
model_consensus_errors.append(int(agent3_consensus_error.loc[12]['permalink']))

In [60]:
inspect_row(agent3_consensus_error, idx=12)

### Модели не хватило информации

In [61]:
model_consensus_errors.append(int(agent3_consensus_error.loc[13]['permalink']))

In [62]:
inspect_row(agent3_consensus_error, idx=13)

### Ничего не найдено в запросе 
(раньше попадали строки с missing)

In [63]:
model_consensus_errors.append(int(agent3_consensus_error.loc[14]['permalink']))

In [64]:
inspect_row(agent3_consensus_error, idx=14)

Учтены все ключевые факторы в запросе - ошибка асессора! 

In [65]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[15]['permalink']))

In [66]:
inspect_row(agent3_consensus_error, idx=15)

Спорный вопрос о ценах. Всё же релевантность какая-то есть: всякие отзывы есть на этот счёт. 

In [67]:
model_consensus_errors.append(int(agent3_consensus_error.loc[16]['permalink']))

In [68]:
inspect_row(agent3_consensus_error, idx=16)

Сложно решить 

In [69]:
model_consensus_errors.append(int(agent3_consensus_error.loc[17]['permalink']))

In [70]:
inspect_row(agent3_consensus_error, idx=17)

### асессор не учитывает локацию в запросе. 
Учтена локация 'тц океания' в запросе

In [71]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[18]['permalink']))

In [72]:
inspect_row(agent3_consensus_error, idx=18)

### Латентная релевантность раскрыта агентом

В поисковике найдена необходимая информация

In [73]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[19]['permalink']))

In [74]:
inspect_row(agent3_consensus_error, idx=19)

In [75]:
model_consensus_errors.append(int(agent3_consensus_error.loc[20]['permalink']))

In [76]:
inspect_row(agent3_consensus_error, idx=20)

In [77]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[21]['permalink']))

In [78]:
inspect_row(agent3_consensus_error, idx=21)

### ничего не найдено в запросе

До корорекции поискового узла в результат поиска попоадали строки с Missing: (исправлено в 3 версии агента)

In [79]:
model_consensus_errors.append(int(agent3_consensus_error.loc[22]['permalink']))

In [80]:
inspect_row(agent3_consensus_error, idx=22)

### Справочный/агрегирующий запрос

Модель восприняла частичное тематическое совпадение как достаточное; не распознала ключевой объект интереса — "сушилка"

In [81]:
model_consensus_errors.append(int(agent3_consensus_error.loc[23]['permalink']))

In [82]:
inspect_row(agent3_consensus_error, idx=23)

### Латентная релевантность раскрыта агентом

In [83]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[24]['permalink']))

In [84]:
inspect_row(agent3_consensus_error, idx=24)

In [85]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[25]['permalink']))

In [86]:
inspect_row(agent3_consensus_error, idx=25)

Все признаки из запроса учтены - ошибка асессора

In [87]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[26]['permalink']))

In [88]:
inspect_row(agent3_consensus_error, idx=26)

In [89]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[27]['permalink']))

### асессор не учитывает локацию в запросе

Модель учла локацию - ошибка асессора

In [90]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[27]['permalink']))

In [91]:
inspect_row(agent3_consensus_error, idx=27)

Модель все признаки учла - ошибка асессора

In [92]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[28]['permalink']))

In [93]:
inspect_row(agent3_consensus_error, idx=28)

### Латентная релевантность раскрыта агентом

In [94]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[29]['permalink']))

In [95]:
inspect_row(agent3_consensus_error, idx=29)

### Латентная релевантность раскрыта агентом

In [96]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[30]['permalink']))

In [97]:
inspect_row(agent3_consensus_error, idx=30)

### асессор не учитывает локацию в запросе

In [98]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[31]['permalink']))

In [99]:
inspect_row(agent3_consensus_error, idx=31)

### Модель упустила важный модификатор в запросе (ошибка модели)

указание на часы работы
похож на справочный\агрегирующий вопрос

In [100]:
model_consensus_errors.append(int(agent3_consensus_error.loc[32]['permalink']))

In [101]:
inspect_row(agent3_consensus_error, idx=32)

### Обрывок\неосмысленный запрос

In [102]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[33]['permalink']))

In [103]:
inspect_row(agent3_consensus_error, idx=33)

Здесь учтены все неоценочные факторы. Точно имеет какую-то релевантность

In [104]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[34]['permalink']))

In [105]:
inspect_row(agent3_consensus_error, idx=34)

Учтены все ключевые факторы

In [106]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[35]['permalink']))

In [107]:
inspect_row(agent3_consensus_error, idx=35)

### Малоинформативный/общий запрос пользователя

С этим типом запросов как правило разметка в сторону 0.0, а модель склонна ставить релевантность. В целом какая-то релевантность таки есть

In [108]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[36]['permalink']))

In [109]:
inspect_row(agent3_consensus_error, idx=36)

Пивнушка Дурдин была по другому адресу и это не метро Сокол.

In [110]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[37]['permalink']))

In [111]:
inspect_row(agent3_consensus_error, idx=37)

### асессор не учитывает локацию в запросе
Локация в запросе не совпадает с локацией организации. 

In [112]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[38]['permalink']))

In [113]:
inspect_row(agent3_consensus_error, idx=38)

### асессор не учитывает локацию в запросе. 
Локация в запросе не совпадает с локацией организации

In [114]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[39]['permalink']))

In [115]:
inspect_row(agent3_consensus_error, idx=39)

In [116]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[40]['permalink']))

In [117]:
inspect_row(agent3_consensus_error, idx=40)

Спорный случай.

In [118]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[41]['permalink']))

In [119]:
inspect_row(agent3_consensus_error, idx=41)

### асессор не учитывает локацию в запросе
Локация в запросе не совпадает с локацией организации

In [120]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[42]['permalink']))

In [121]:
inspect_row(agent3_consensus_error, idx=42)

### ошибка модели

Как будто бы сревиса Volvo нет по этому адресу нет

In [122]:
model_consensus_errors.append(int(agent3_consensus_error.loc[32]['permalink']))

In [123]:
inspect_row(agent3_consensus_error, idx=43)

### асессор не учитывает локацию в запросе

In [124]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[44]['permalink']))

In [125]:
inspect_row(agent3_consensus_error, idx=44)

In [126]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[45]['permalink']))

In [127]:
inspect_row(agent3_consensus_error, idx=45)

In [128]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[46]['permalink']))

In [129]:
inspect_row(agent3_consensus_error, idx=46)

### ошибка модели

Вот тут тот случай, когда упоминание Люберцы из адреса вносит неуверенность модели, потому что в запросе упоминаетмя котельники и модель может путаться

In [130]:
model_consensus_errors.append(int(agent3_consensus_error.loc[47]['permalink']))

In [131]:
inspect_row(agent3_consensus_error, idx=47)

### ошибка модели

In [132]:
model_consensus_errors.append(int(agent3_consensus_error.loc[48]['permalink']))

In [133]:
inspect_row(agent3_consensus_error, idx=48)

### асессор не учитывает локацию в запросе
Локация в запросе не совпадает с локацией организации

In [134]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[49]['permalink']))

In [135]:
inspect_row(agent3_consensus_error, idx=49)

In [136]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[50]['permalink']))

In [137]:
inspect_row(agent3_consensus_error, idx=50)

### ошибка модели

In [138]:
model_consensus_errors.append(int(agent3_consensus_error.loc[51]['permalink']))

In [139]:
inspect_row(agent3_consensus_error, idx=51)

In [140]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[52]['permalink']))

In [141]:
inspect_row(agent3_consensus_error, idx=52)

### ошибка модели

In [142]:
model_consensus_errors.append(int(agent3_consensus_error.loc[51]['permalink']))

In [143]:
inspect_row(agent3_consensus_error, idx=53)

### Малоинформативный/общий запрос пользователя

В целом Бургер Кинг Авто - это еда...Поэтому запрос нельзя считать абсолютно нерелевантным

Спорный случай

In [144]:
model_consensus_errors.append(int(agent3_consensus_error.loc[54]['permalink']))

In [145]:
inspect_row(agent3_consensus_error, idx=54)

In [146]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[55]['permalink']))

In [147]:
inspect_row(agent3_consensus_error, idx=55)

### асессор не учитывает локацию в запросе
Локация в запросе не совпадает с локацией организации

In [148]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[56]['permalink']))

In [149]:
inspect_row(agent3_consensus_error, idx=56)

### асессор не учитывает локацию в запросе

In [150]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[57]['permalink']))

In [151]:
inspect_row(agent3_consensus_error, idx=57)

In [152]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[58]['permalink']))

In [153]:
inspect_row(agent3_consensus_error, idx=58)

В отзывах есть информация для ответа

In [154]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[59]['permalink']))

In [155]:
inspect_row(agent3_consensus_error, idx=59)

Организация занимается шиномонтажем

In [156]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[60]['permalink']))

In [157]:
inspect_row(agent3_consensus_error, idx=60)

### асессор не учитывает локацию в запросе
Локация в запросе не совпадает с локацией организации

In [158]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[61]['permalink']))

In [159]:
inspect_row(agent3_consensus_error, idx=61)

In [160]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[62]['permalink']))

In [161]:
inspect_row(agent3_consensus_error, idx=62)

### ошибка модели
нет подтверждения информации об изготовлении штор

In [162]:
model_consensus_errors.append(int(agent3_consensus_error.loc[63]['permalink']))

In [163]:
inspect_row(agent3_consensus_error, idx=63)

### ошибка модели

In [164]:
model_consensus_errors.append(int(agent3_consensus_error.loc[64]['permalink']))

In [165]:
inspect_row(agent3_consensus_error, idx=64)

In [166]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[65]['permalink']))

In [167]:
inspect_row(agent3_consensus_error, idx=65)

### Малоинформативный/общий запрос пользователя
В целом отвечает запросу и клиника не закрыта

In [168]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[66]['permalink']))

In [169]:
inspect_row(agent3_consensus_error, idx=66)

In [170]:
assessors_errors_consensus.append(int(agent3_consensus_error.loc[67]['permalink']))

In [171]:
inspect_row(agent3_consensus_error, idx=67)

In [172]:
total_unique = len(agent3_consensus_error)
num_model_errors = len(model_consensus_errors)
num_assessor_errors = len(assessors_errors_consensus)

print(f"Количество консенсусных ошибок: {total_unique}")
print(f"Из них модельных ошибок: {num_model_errors} ({num_model_errors / total_unique:.1%})")
print(f"Ошибок аннотаций: {num_assessor_errors} ({num_assessor_errors / total_unique:.1%})")

# Проверка: пересекаются ли model_errors и assessors_errors (не должно быть)
intersection = set(model_consensus_errors) & set(assessors_errors_consensus)
if intersection:
    print(f"⚠️ Пересечения в списках model_errors и assessors_errors: {len(intersection)}")

Количество консенсусных ошибок: 68
Из них модельных ошибок: 22 (32.4%)
Ошибок аннотаций: 47 (69.1%)


# Выводы

In [173]:
acc_df

,split,model,description,accuracy
0,val,baseline,gpt + few-shot in-context learning prompt,0.6856
1,val,agent1,агент 1 версия,0.6957
2,val,agent2,агент 2 версия,0.6221
3,val,agent3,агент 3 версия,0.6388
4,test,baseline,gpt + few-shot in-context learning prompt,0.7840
5,test,agent1,агент 1 версия,0.7640
6,test,agent2,агент 2 версия,0.7360
7,test,agent3,агент 3 версия,0.7440


In [174]:
# Общая статистика
total_all = len(unique_error_agent3)
model_all = len(model_errors_3)
annot_all = len(assessors_errors_3)
intersect_all = set(model_errors_3) & set(assessors_errors_3)

# Консенсусная выборка (вся)
total_consensus = len(agent3_consensus_error)
model_consensus = len(model_consensus_errors)
annot_consensus = len(assessors_errors_consensus)
intersect_consensus = set(model_consensus_errors) & set(assessors_errors_consensus)

# Сводная таблица
df = pd.DataFrame({
    "Всего ошибок": [total_all, total_consensus],
    "Ошибки агента": [model_all, model_consensus],
    "% агентных ошибок": [f"{model_all / total_all:.1%}", f"{model_consensus / total_consensus:.1%}"],
    "Ошибки аннотаций": [annot_all, annot_consensus],
    "% аннотационных ошибок": [f"{annot_all / total_all:.1%}", f"{annot_consensus / total_consensus:.1%}"],
    "Пересечения": [len(intersect_all), len(intersect_consensus)]
}, index=["Количество уникальных ошибок агента 3", "Количество консенсусных ошибок"])

df

,Всего ошибок,Ошибки агента,% агентных ошибок,Ошибки аннотаций,% аннотационных ошибок,Пересечения
Количество уникальных ошибок агента 3,9,6,66.7%,3,33.3%,0
Количество консенсусных ошибок,68,22,32.4%,47,69.1%,0


In [175]:
balance_table

relevance_new,1.0,0.0
распределение таргета на валидации,57.53,42.47
распределение таргета на тесте,63.40,36.60


In [176]:
pred_balance_table

pred_relevance,1.0,0.0
распределение таргета на валидации,66.22,33.78
распределение таргета на тесте,70.20,29.80


In [177]:
def count_fp_fn(df):
    fp = ((df["pred_relevance"] == 1.0) & (df[RELEVANCE_COL] == 0.0)).sum()
    fn = ((df["pred_relevance"] == 0.0) & (df[RELEVANCE_COL] == 1.0)).sum()
    return fp, fn

fp_all, fn_all = count_fp_fn(unique_error_agent3)
fp_consensus, fn_consensus = count_fp_fn(agent3_consensus_error)

df_fp_fn = pd.DataFrame({
    "False Positives": [fp_all, fp_consensus],
    "False Negatives": [fn_all, fn_consensus],
    "Всего ошибок": [len(unique_error_agent3), len(agent3_consensus_error)]
}, index=["Уникальные ошибки Agent3", "Консенсусные ошибки"])

df_fp_fn["% FP"] = [f"{fp / total:.1%}" for fp, total in zip(df_fp_fn["False Positives"], df_fp_fn["Всего ошибок"])]
df_fp_fn["% FN"] = [f"{fn / total:.1%}" for fn, total in zip(df_fp_fn["False Negatives"], df_fp_fn["Всего ошибок"])]
df_fp_fn

,False Positives,False Negatives,Всего ошибок,% FP,% FN
Уникальные ошибки Agent3,5,4,9,55.6%,44.4%
Консенсусные ошибки,47,21,68,69.1%,30.9%


В Agent2 в промте по внешнему поиску допущена 1 логическая ошибка, поэтому не комментирую результаты этого агента и не провожу сравнения с ним. 

Agent1 выигрывает на валидации (0.6957 vs 0.6288), но:

* Судя по ошибкам, валидация шумная

* В консенсусных ошибках 69.1%  - аннотационных. Это очень высокий уровень шума, то есть "реальных" ошибок меньше.

В уникальных ошибках на валидации Agent3 — 3 из 9 аннотационных. На тесте же уникальных ошибок у агента 3 меньше, чем у агента 1 (13 vs 17). Доверие к приросту от Agent1 по сравнению с Agent3, особенно при небольшой разнице на шумной валидации и на тесте, снижается.

Судя по тесту бейзлайн не сильно лучше агента, а чуть выше по цифре (0.7840 vs 0.7640), но это неустойчиво при наличии шума. В консенсусных ошибках: 69.1% — ошибки аннотаций, то есть "большинство" ошибок - не вина агентов и мы не может утверждать, что агенты ошибаются сильнее, чем бейзлайн. 

Данные также не подтверждают, что сильный бейзлайн надёжнее, агенты ошибаются "иначе", но не сильно хуже и способны вскрывать "скрытую" релевантность\нерелевантность при помощи внешнего поиска в Интернете, то есть точно есть потенциал быть лучше. Однако "побить" бейзлайн в данном проекте мне не удалось. Думаю, мой агент мог бы дать какой-то прирост, но не при текущей разметке — она слишком шумная для точной (более тонкой) оценки. (В разметке примерно 26% ошибок, связанных с тем, что асессор не учитывает локацию в запросе, также есть ошибки, связанные с тем, что не учитывается информация в отзывах.)

Отмечу возможные направления для улучшения агента: 

* направлять внешний поиск информации при появлении указания на время работы организации (круглосуточный, 24 часа, "24/7"..)
* подчеркнуть важность модификаторов формата услуг в запросе типа "дежурный", "на колесах", "с выездом", "выездной", "частный", "стейк ресторан" и т.п.